here's the link if you want to test: https://www.kaggle.com/code/nlordqting/phobertv2-vietnamese-sentiment-analysis

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evaluate
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install underthesea


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tk79619_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tk79619_
  Resolved https://github.com/huggingface/transformers.git to commit d522afea1324b8156c929f3896df14762c9ea716
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8661673 sha256=008fde85b5c2bf36ad3a5749b4ab370faadbc04329687fffb8f74cf5f798d6af
  Stored in directory: /tmp/pip-ephem-wheel-cache-61e5919d/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
 

In [3]:
from underthesea import word_tokenize

text = "Tôi là sinh viên."
words = word_tokenize(text)
print(words)


['Tôi', 'là', 'sinh viên', '.']


In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import EvalPrediction
from evaluate import load
import numpy as np
import re

# Preprocessing functions
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove URLs
    text = re.sub(r"https://\S+|www\.\S+", '', text)
    # Remove emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

# Custom Dataset for Vietnamese Sentiment Analysis
class VietnameseSentimentAnalysisDataset(Dataset):
    def __init__(self, directory, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = []
        self.labels = []
        
        # Load positive and negative files
        for label, sentiment in enumerate(["neg", "pos"]):
            sentiment_dir = os.path.join(directory, sentiment)
            for filename in os.listdir(sentiment_dir):
                file_path = os.path.join(sentiment_dir, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read().strip()
                    # Preprocess text
                    text = preprocess_text(text)
                    self.texts.append(text)
                    self.labels.append(label)
                    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Use underthesea for word segmentation
        segmented_text = word_tokenize(text)
        # Tokenize and encode the text
        tokenized_text = self.tokenizer(" ".join(segmented_text), max_length=self.max_length, padding='max_length', truncation=True)
        
        # Convert to torch tensor
        item = {key: torch.tensor(val) for key, val in tokenized_text.items()}
        item['labels'] = torch.tensor(label)
        return item

# Initialize tokenizer with BARTpho
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Example dataset initialization
train_dataset = VietnameseSentimentAnalysisDataset(
    directory="/kaggle/input/vietnamese-sentiment-analysis-lab-thay-huong/data_train/data_train/train", 
    tokenizer=tokenizer
)

val_dataset = VietnameseSentimentAnalysisDataset(
    directory="/kaggle/input/vietnamese-sentiment-analysis-lab-thay-huong/data_train/data_train/test", 
    tokenizer=tokenizer
)

test_dataset = VietnameseSentimentAnalysisDataset(
    directory="/kaggle/input/vietnamese-sentiment-analysis-lab-thay-huong/data_test/data_test/test", 
    tokenizer=tokenizer
)

2024-03-13 06:44:58.047956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 06:44:58.048065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 06:44:58.206975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

# Custom trainer 


Insprired by https://www.kaggle.com/code/minhcng/sentiment-analysis-using-roberta-lstm

In [5]:
for i in range(5):
    sample = train_dataset[i]
    print(f"Sample {i+1}:")
    print(f"Text: {tokenizer.decode(sample['input_ids'], skip_special_tokens=True)}")
    print(f"Label: {sample['labels'].item()}")
    print("")

Sample 1:
Text: this place is quite nice flan gato is acceptable and almost the others are unauthentic ones i suppuse i called a tiramisu with chocolate flavor and it was the worst tiramisu i ve ever eaten anyhow i recommend this store for everyone who needs a place for hanging out with some sweetie cake
Label: 0

Sample 2:
Text: đă ăn và thấy cũng tạm tạm nhưng đặc_biệt là không thích thái độ phục_vụ của nhân_viên trong quán kêu 2 3 lần tô cháo mà nhìn rồi quay mặt đi giống như mình đang đi xin
Label: 0

Sample 3:
Text: lần đầu được chở đi ăn_ở đây là lúc sinh_nhật con bạn chẳng hiểu là do ngon hay vui mà lúc đó thấy món nào cũng ngon chỉ có mì jajang hơi mặn chút trưa này có dịp đi ngang ghé vào ăn lần hai thì hơi gượng lúc bước vào nghe bảo mình đi một_mình thì bạm nb có_vẻ khựng lại vào kêu 1 tokk ramen vs 1 cơm bokkum cơm thì lần đầu ăn ngon lắm cơm nóng_hổi có cả lớp cơm cháy phía dưới còn hôm_nay thì nguội_ngắt còn hơi nhạt nữa về phần tokk thì ko còn gì để nói ngoài thấy ghê có

In [6]:
# from evaluate import load
# import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    # Load metrics
    precision_metric = load("precision")
    recall_metric = load("recall")
    f1_metric = load("f1")
    accuracy_metric = load("accuracy")
    
    # Compute metrics
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


In [7]:
from transformers import TrainingArguments, Trainer

class SentimentTrainer:
    def __init__(self, config):
        self.config = config
        self.model = AutoModelForSequenceClassification.from_pretrained(config.pretrained_path, num_labels=config.model_num_labels)
        self.tokenizer = AutoTokenizer.from_pretrained(config.pretrained_path)
        
        # Load datasets
        self.train_dataset, self.eval_dataset = self.load_dataset()
        
        # Define TrainingArguments
        training_args = TrainingArguments(
            output_dir=config.training_output_dir,
            evaluation_strategy="steps",
            eval_steps=config.training_eval_steps,  # Evaluate every eval_steps
            per_device_train_batch_size=config.training_batch_size,
            per_device_eval_batch_size=config.training_batch_size,
            num_train_epochs=config.training_num_epochs,
            save_steps=config.training_save_steps,
            logging_dir=config.training_logging_dir,
            logging_steps=config.training_logging_steps,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            report_to="none",  # Disable wandb
        )
        
        # Initialize the Trainer
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset= val_dataset,
            tokenizer=self.tokenizer,
            compute_metrics=compute_metrics  # Use the compute_metrics function
        )
    
    def load_dataset(self):
#         self.train_dataset = create_sentiment_dataset(self.config.train_data_path, self.tokenizer, "train")
#         self.eval_dataset = create_sentiment_dataset(self.config.eval_data_path, self.tokenizer, "eval")
        self.train_dataset =  train_dataset
        self.eval_dataset =  val_dataset
        return self.train_dataset, self.eval_dataset

    
    def train(self):
        self.trainer.train()

    def eval(self):
        evaluation_results = self.trainer.evaluate(eval_dataset=self.eval_dataset)
        print(evaluation_results)
        return evaluation_results



In [8]:
class SentimentConfig:
    def __init__(self):
        self.pretrained_path = "vinai/phobert-base-v2"
        self.model_num_labels = 2  # Assuming binary classification
        self.training_device = "cuda" if torch.cuda.is_available() else "cpu"
        self.training_output_dir = "/kaggle/working/results"
        self.training_eval_steps = 50  # Evaluate every 50 steps
        self.training_batch_size = 16
        self.training_num_epochs = 1
        self.training_save_steps = 1000
        self.training_logging_dir = "/kaggle/working/logs"
        self.training_logging_steps = 100

# Create an instance of this configuration
config = SentimentConfig()

# Train the model

In [9]:

trainer = SentimentTrainer(config)
trainer.train()


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_b

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,No log,0.317216,0.884579,0.871700,0.870617,0.871700
100,0.383200,0.332640,0.888670,0.873800,0.872581,0.873800
150,0.383200,0.273030,0.897133,0.896100,0.896032,0.896100
200,0.309900,0.291478,0.897332,0.888200,0.887554,0.888200
250,0.309900,0.269622,0.898246,0.898200,0.898197,0.898200
300,0.278800,0.242169,0.909118,0.905300,0.905079,0.905300
350,0.278800,0.254306,0.907262,0.906900,0.906879,0.906900
400,0.253800,0.252348,0.911169,0.910100,0.910042,0.910100
450,0.253800,0.260174,0.897082,0.894200,0.894008,0.894200
500,0.252300,0.258773,0.905170,0.904400,0.904355,0.904400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

# Test the model

In [10]:
#Make predictions
predictions, label_ids, metrics = trainer.trainer.predict(test_dataset=test_dataset)

# Optionally, compute metrics based on predictions and labels
# This assumes you have a way to calculate metrics from predictions, similar to compute_metrics
test_metrics = compute_metrics((predictions, label_ids))

print("Test Metrics:", test_metrics)

# def manual_predict(model, dataloader, device):
#     model.to(device)
#     model.eval()
#     predictions = []
#     with torch.no_grad():
#         for batch in dataloader:
#             inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
#             outputs = model(**inputs)
#             logits = outputs.logits
#             predictions.append(logits.argmax(dim=-1).cpu().numpy())
#     return np.concatenate(predictions)

# # Adjust the batch size for the DataLoader
# test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)  # Smaller batch size

# # Assuming your model is stored in `trainer.model`
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# predictions = manual_predict(trainer.model, test_dataloader, device)

Test Metrics: {'precision': 0.9220492278219332, 'recall': 0.922, 'f1': 0.9219977254536741, 'accuracy': 0.922}


# Save the model

In [11]:
# Save the model and tokenizer
model_save_path = "/kaggle/working/sentiment_model"
trainer.model.save_pretrained(model_save_path)
trainer.tokenizer.save_pretrained(model_save_path)


('/kaggle/working/sentiment_model/tokenizer_config.json',
 '/kaggle/working/sentiment_model/special_tokens_map.json',
 '/kaggle/working/sentiment_model/vocab.txt',
 '/kaggle/working/sentiment_model/bpe.codes',
 '/kaggle/working/sentiment_model/added_tokens.json')

# Load the model for inference if you want

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the trained model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# Ensure the model is in evaluation mode
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

# Performing Inference on Test Samples


In [13]:
import torch

def predict(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return predictions

# Example sentences from test set: sentence 1 and 2 has "neg" label and sentence 3 has "pos" label
test_sentences = ["""
Quán này khá là nổi_tiếng nay mới có dịp ghé thử . Năm ở vii trí khá dễ tìm , quán hơi nhỏ , đi chiều_tối là đông lắm phải đợi lâu
Nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm
Món ăn :
Mình gọi gà sốt phô_mai cay : gà quá bở , thịt ăn vào k thơm , món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy . Cảm_giác ngán lắm
Cơm trộn kim_chi HQ thì_phải : ăn hơi cay , nóng_hổi luôn , vị ăn ok nhưng k đặc_sắc . Với giá đó thì ăn k đáng tiền .
Khoai_tây chiên : dở và ăn vị k ngon
Tóm_lại k như kì_vọng của mình , giá_cả ở đây quá đắt so với chất_lượng . Mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt
À có nước_lọc miễn_phí nha . Mọi người tới ăn nhớ hỏi ."""
                  ,"""Chưa có nhìu món về HQ , vẫn mang đậm_nét ẩm_thực của quán Việt ( tên thì rất Korea ) , thực_đơn sơ_sài ... hi_vọng sẽ cập_nhật nhìu món ăn kiểu " vỉa_hè " mang đậm chất Korea"""
                  , "An ngon , gia re , sach se mat me con gi bang"
            
                 ]

# Get predictions
predictions = predict(test_sentences)

# Process predictions
for sentence, prediction in zip(test_sentences, predictions):
    print(f"Sentence: {sentence}")
    print(f"Prediction: {prediction.numpy()}")
    print("This is a positive review" if prediction.argmax() == 1 else "This is a negative review")


Sentence: 
Quán này khá là nổi_tiếng nay mới có dịp ghé thử . Năm ở vii trí khá dễ tìm , quán hơi nhỏ , đi chiều_tối là đông lắm phải đợi lâu
Nhân_viên thì bình_thường à vì do đông quá nên cũng thông_cảm
Món ăn :
Mình gọi gà sốt phô_mai cay : gà quá bở , thịt ăn vào k thơm , món này đã có mỡ rồi cộng thêm da_gà nữa ăn rất ngáy . Cảm_giác ngán lắm
Cơm trộn kim_chi HQ thì_phải : ăn hơi cay , nóng_hổi luôn , vị ăn ok nhưng k đặc_sắc . Với giá đó thì ăn k đáng tiền .
Khoai_tây chiên : dở và ăn vị k ngon
Tóm_lại k như kì_vọng của mình , giá_cả ở đây quá đắt so với chất_lượng . Mà đi ăn phải chầu_chực như_thế thì cũng hơi mệt
À có nước_lọc miễn_phí nha . Mọi người tới ăn nhớ hỏi .
Prediction: [0.9952714 0.0047286]
This is a negative review
Sentence: Chưa có nhìu món về HQ , vẫn mang đậm_nét ẩm_thực của quán Việt ( tên thì rất Korea ) , thực_đơn sơ_sài ... hi_vọng sẽ cập_nhật nhìu món ăn kiểu " vỉa_hè " mang đậm chất Korea
Prediction: [0.97268885 0.02731115]
This is a negative review
Sentence

# 

Not bad!!!